In [ ]:
!pip install SpeechRecognition pyaudio spacy
!python -m spacy download en_core_web_sm

In [2]:
with open("voice_recognition.py", "w") as file:
    file.write('''
import streamlit as st
import speech_recognition as sr
import nltk
import spacy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from spacy import displacy

# Initialize Spacy NLP
nlp = spacy.load("en_core_web_sm")

# Set up NLTK
nltk.download('punkt')
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

# Initialize Speech Recognizer
recognizer = sr.Recognizer()

def recognize_speech():
    # Use the microphone as the audio source
    with sr.Microphone() as source:
        st.info("Listening... Please speak clearly.")
        recognizer.adjust_for_ambient_noise(source)  # Adjust for background noise
        audio = recognizer.listen(source)
        
        try:
            # Convert speech to text
            text = recognizer.recognize_google(audio)
            st.success(f"Recognized Speech: {text}")
            return text
        except sr.UnknownValueError:
            st.error("Could not understand the audio.")
        except sr.RequestError as e:
            st.error(f"Could not request results; {e}")
    
    return None

def process_text(text):
    # Tokenize and Remove Stop Words
    tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in tokens if word.isalnum() and word not in stop_words]
    
    # Display Tokenization Result
    st.subheader("Tokenized Words (without stopwords):")
    st.write(filtered_tokens)
    
    # Perform Named Entity Recognition (NER) with Spacy
    doc = nlp(text)
    st.subheader("Named Entity Recognition:")
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    st.write(entities)
    
    # Visualize the Entities
    st.subheader("Entity Visualization:")
    html = displacy.render(doc, style="ent")
    st.write(html, unsafe_allow_html=True)

# Streamlit Web App Interface
st.title("Speech Recognition with NLP")
st.write("Click the button below and speak to see speech-to-text and some basic NLP processing.")

if st.button("Start Speech Recognition"):
    recognized_text = recognize_speech()
    if recognized_text:
        process_text(recognized_text)

''')